6409 Чернышова Дана

# Введение в MapReduce модель на Python


In [48]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [49]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [50]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [51]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [52]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [53]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [54]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [55]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [56]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [57]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [58]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [59]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [60]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [61]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [62]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.8245603783026083),
 (1, 1.8245603783026083),
 (2, 1.8245603783026083),
 (3, 1.8245603783026083),
 (4, 1.8245603783026083)]

## Inverted index

In [63]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [64]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [65]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [66]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('banana', 2), ('is', 18), ('it', 18)]),
 (1, [('a', 2), ('what', 10)])]

## TeraSort

In [67]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.07534984844604264),
   (None, 0.11575428867591808),
   (None, 0.11823822285746755),
   (None, 0.12388922749259268),
   (None, 0.12890352184823684),
   (None, 0.13265864133591931),
   (None, 0.15112732846741905),
   (None, 0.17968388066527885),
   (None, 0.18356653573650383),
   (None, 0.1911976877063578),
   (None, 0.1935178109113702),
   (None, 0.23224343748896992),
   (None, 0.2489212269347667),
   (None, 0.2759351289748877),
   (None, 0.31379593567594044),
   (None, 0.3262519765970081),
   (None, 0.3548461456222435),
   (None, 0.37370437782253907),
   (None, 0.38244548975141934),
   (None, 0.39563560910236384),
   (None, 0.4421895591124001)]),
 (1,
  [(None, 0.6332238447749602),
   (None, 0.7707109538383637),
   (None, 0.7758896236812959),
   (None, 0.7893081213166308),
   (None, 0.8469678957609256),
   (None, 0.8877078130348477),
   (None, 0.9310397828871886),
   (None, 0.9777599160027518),
   (None, 0.9804231614103326)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [68]:
from typing import Iterator
import numpy as np

def RECORDREADER(size: int):
    data = [(index, num) for (index, num) in enumerate(np.random.randint(1000, size=size))]
    print(data)
    return data

def MAP(_, number):
    yield (None, number)

def REDUCE(_, numbers: Iterator[int]):
    max_number = max(numbers)
    yield max_number

output = MapReduce(lambda: RECORDREADER(10), MAP, REDUCE)
max_number = list(output)
max_number

[(0, 998), (1, 10), (2, 893), (3, 489), (4, 737), (5, 886), (6, 364), (7, 562), (8, 281), (9, 38)]


[998]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [69]:
from typing import Iterator

def RECORDREADER(size):
    data = [(index, num) for (index, num) in enumerate(np.random.randint(1000, size=size))]
    print(data)
    return data

def MAP(_, number):
    yield (1, number)

def REDUCE(_, numbers: Iterator[int]):
    total_sum = 0
    count = 0
    for num in numbers:
        total_sum += num
        count += 1
    if count > 0:
        average = total_sum / count
        yield average

output = MapReduce(lambda: RECORDREADER(10), MAP, REDUCE)
output = list(output)
output

[(0, 596), (1, 995), (2, 974), (3, 873), (4, 234), (5, 6), (6, 457), (7, 667), (8, 545), (9, 467)]


[581.4]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [70]:
def groupbykey_sorted(iterable):
    # sort the input data by the key
    sorted_iterable = sorted(iterable, key=lambda x: x[0])
    result = [] 
    current_key = None
    for key, value in sorted_iterable:
        if key != current_key:
            if current_key is not None:
                result.append((current_key, sorted(current_group)))
            current_key = key  # update the current key
            current_group = [value]
        else:
            current_group.append(value)
    if current_key is not None:
        result.append((current_key, sorted(current_group)))
    return result

def MAP(key: int, value: int):
    yield (1, value)

def RECORDREADER():
    numbers = [(index, num) for (index, num) in enumerate(np.random.randint(1000, size=10))]
    print(numbers)
    return numbers

print(groupbykey_sorted(flatten(map(lambda x: MAP(*x), RECORDREADER()))))

[(0, 175), (1, 791), (2, 65), (3, 894), (4, 455), (5, 756), (6, 630), (7, 52), (8, 482), (9, 741)]
[(1, [52, 65, 175, 455, 482, 630, 741, 756, 791, 894])]


# Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [71]:
def RECORDREADER(size):
    data = [(index, value) for (index, value) in enumerate(np.random.randint(10, size=size))]
    print(data, '\n')
    return data

def MAP(_, value):
    yield (value, value)

def REDUCE(key, _):
    yield (key)

output = MapReduce(lambda: RECORDREADER(10), MAP, REDUCE)
output = list(output)
print(output)

[(0, 3), (1, 9), (2, 3), (3, 6), (4, 4), (5, 9), (6, 2), (7, 2), (8, 5), (9, 2)] 

[3, 9, 6, 4, 2, 5]


#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.

In [72]:
db_data = [
    {'id': 1, 'name': 'John', 'age': 25},
    {'id': 2, 'name': 'Alice', 'age': 25},
    {'id': 3, 'name': 'Bob', 'age': 35}
]

def RECORDREADER():
    for record in db_data:
        yield (record['id'], record)

def MAP(_, record):
    if record['age'] == 25: 
        yield (record['id'], record)

def REDUCE(key, tuples):
    for record in tuples:
        yield record 

output = MapReduce(RECORDREADER, MAP, REDUCE)

selected_values = list(output)
print(selected_values)

[{'id': 1, 'name': 'John', 'age': 25}, {'id': 2, 'name': 'Alice', 'age': 25}]


In [73]:
from typing import NamedTuple

class Record(NamedTuple):
    id: int
    name: str
    age: int

def RECORDREADER():
    for record in db_data:
        yield (record.id, record)

def MAP(_, record):
    if record.age == 25:
        yield record, record 

db_data = [
    Record(id = 1, name = 'John', age = 25),
    Record(id = 2, name = 'Alice', age = 25),
    Record(id = 3, name = 'Bob', age = 35)
]

output = MapReduce(RECORDREADER, MAP, REDUCE)

selected_values = [tuple(record) for record in output]
selected_values

[(1, 'John', 25), (2, 'Alice', 25)]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [74]:
db_data = [
    {'id': 1, 'name': 'John', 'age': 25},
    {'id': 2, 'name': 'Alice', 'age': 25},
    {'id': 3, 'name': 'Bob', 'age': 35}
]

def MAP(_, record):
    attributes_to_keep = ['id', 'name']
    projected_record = tuple(record[attr] for attr in attributes_to_keep)
    yield (record['id'], projected_record)

def REDUCE(key, records):
    yield records[0]

def RECORDREADER():
    for record in db_data:
        yield (record['id'], record)

output = MapReduce(RECORDREADER, MAP, REDUCE)

selected_values = list(output)
selected_values

[(1, 'John'), (2, 'Alice'), (3, 'Bob')]

In [75]:
from typing import NamedTuple

class Record(NamedTuple):
    id: int
    name: str
    age: int

db_data = [
    Record(id=1, name='John', age=25),
    Record(id=2, name='Alice', age=25),
    Record(id=3, name='Bob', age=35)
]

def RECORDREADER():
    for record in db_data:
        yield (record.id, record)

def MAP(_, record):
    projected_record = (record.id, record.age)
    yield (record.id, projected_record)

def REDUCE(key, records):
    for _, projected_record in records:
        yield projected_record

output = MapReduce(RECORDREADER, MAP, REDUCE)

selected_values = list(output)
selected_values

[25, 25, 35]

In [76]:
from typing import NamedTuple
from collections import defaultdict

class Record(NamedTuple):
    id: int
    name: str
    age: int

db_data = [
    Record(id=1, name='John', age=25),
    Record(id=2, name='Alice', age=25),
    Record(id=3, name='Bob', age=35)
]

def MAP(_, record):
    yield (record.id, record.name)

def REDUCE(key, records):
    yield (key, records[0])

def RECORDREADER():
    for record in db_data:
        yield record.id, record

output = MapReduce(RECORDREADER, MAP, REDUCE)

selected_values = list(output)
selected_values

[(1, 'John'), (2, 'Alice'), (3, 'Bob')]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [77]:
def RECORDREADER():
    for record in table_R + table_S:
        yield (None, record)

def MAP(_, record):
    yield (record, record)

def REDUCE(key, values):
    for value in set(values):
        yield (value)

table_R = [
    (1, 'John'),
    (2, 'Alice'),
    (5, 'Bob')
]

table_S = [
    (1, 'John'),
    (4, 'Los Angeles'),
    (6, 'Chicago')
]

output = MapReduce(RECORDREADER, MAP, REDUCE)
result = list(output)
result

[(1, 'John'), (2, 'Alice'), (5, 'Bob'), (4, 'Los Angeles'), (6, 'Chicago')]

In [78]:
table_R = [
    (1, 'John'),
    (2, 'Alice'),
    (5, 'Bob')
]

table_S = [
    (1, 'John'),
    (4, 'Los Angeles'),
    (6, 'Chicago')
]

def RECORDREADER():
    for record in table_R + table_S:
        yield (None, record)

def MAP(_, record):
    yield (record, record)

def REDUCE(key, values):
    unique_values = set()
    for value in values:
        if value[1] in unique_values:
            continue
        unique_values.add(value[1])
        yield value

output = MapReduce(RECORDREADER, MAP, REDUCE)
result = list(output)
result

[(1, 'John'), (2, 'Alice'), (5, 'Bob'), (4, 'Los Angeles'), (6, 'Chicago')]

In [79]:
from typing import NamedTuple

class Record(NamedTuple):
    id: int
    name: str
    age: int

def RECORDREADER():
    for record in table_R + table_S:
        yield (None, record)

def MAP(_, record):
    yield (record, record)

def REDUCE(key, values):
    for value in set(values):
        yield (value)

table_R = [
    Record(id=1, name='John', age=20),
    Record(id=2, name='Alice', age=25),
    Record(id=3, name='Bob', age=30)
]

table_S = [
    Record(id=4, name='Charlie', age=22),
    Record(id=5, name='David', age=27),
    Record(id=6, name='Emma', age=32)
]

output = MapReduce(RECORDREADER, MAP, REDUCE)
result = list(output)
result = [tuple(record) for record in result]
result

[(1, 'John', 20),
 (2, 'Alice', 25),
 (3, 'Bob', 30),
 (4, 'Charlie', 22),
 (5, 'David', 27),
 (6, 'Emma', 32)]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [80]:
def RECORDREADER():
    for record in table_R + table_S:
        yield (None, record)

def MAP(_, record):
    # map key with the value
    yield (record[1:], record)

def REDUCE(key, values):
    if len(values) == 2 and values[0][1:] == values[1][1:]:
        # if exists => return one of them
        yield (values[0])

table_R = [
    (1, 'John'),
    (2, 'Alice'),
    (3, 'Bob')
]

table_S = [
    (12, 'Alice'),
    (3, 'Bob'),
    (4, 'Charlie')
]

output = MapReduce(RECORDREADER, MAP, REDUCE)

result = list(output)

result

[(2, 'Alice'), (3, 'Bob')]

In [81]:
from typing import NamedTuple

class Record(NamedTuple):
    id: int
    name: str
    age: int

def RECORDREADER():
    for record in table_R + table_S:
        yield (None, Record(id=record[0], name=record[1], age=record[2]))

def MAP(_, record):
    yield ((record.name, record.age), record)

def REDUCE(key, values):
    if len(values) == 2 and values[0][1:] == values[1][1:]:
        yield (values[0][1:])

table_R = [
    Record(id=1, name='John', age=25),
    Record(id=2, name='Alice', age=30),
    Record(id=3, name='Bob', age=35)
]

table_S = [
    Record(id=12, name='Alice', age=10),
    Record(id=3, name='Bob', age=35),
    Record(id=4, name='Charlie', age=40)
]

output = MapReduce(RECORDREADER, MAP, REDUCE)

result = list(output)

result

[('Bob', 35)]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [82]:
def MAP(_, record):
    # create the (t, R) pair for the tuples of R
    # (t, S) pair for the tuples of S
    if record[1] == 'R':
        yield (record[0], 'R')
    elif record[1] == 'S':
        yield (record[0], 'S')

def REDUCE(key, values):
    # if exists only one relation, create (t, t) pair
    if len(values) == 1:
        yield (key)

def RECORDREADER():
    for record in table_R:
        yield (None, (record, 'R'))

    for record in table_S:
        yield (None, (record, 'S'))

table_R = [
    (1, 'John'),
    (2, 'Alice'),
    (3, 'Bob')
]

table_S = [
    (2, 'Alice'),
    (3, 'Bob'),
    (4, 'Charlie')
]

output = MapReduce(RECORDREADER, MAP, REDUCE)
result = list(output)
result

[(1, 'John'), (4, 'Charlie')]

In [83]:
from typing import NamedTuple

class Record(NamedTuple):
    id: int
    name: str

def MAP(_, record):
    if record[1] == 'R':
        yield (record[0].name, 'R')
    elif record[1] == 'S':
        yield (record[0].name, 'S')

def REDUCE(key, values):
    if len(values) == 1:
        yield (key)

def RECORDREADER():
    for record in table_R:
        yield (None, (record, 'R'))

    for record in table_S:
        yield (None, (record, 'S'))

table_R = [
    Record(id=1, name='John'),
    Record(id=2, name='Alice'),
    Record(id=3, name='Bob')
]

table_S = [
    Record(id=12, name='Alice'),
    Record(id=3, name='Bob'),
    Record(id=4, name='Charlie')
]

output = MapReduce(RECORDREADER, MAP, REDUCE)
result = list(output)
print(result)

['John', 'Charlie']


### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [84]:
def MAP(_, record):
    record_id = record['id']
    if 'name' in record:
        yield (record_id, ('R', record))
    elif 'city' in record:
        yield (record_id, ('S', record))

def REDUCE(key, values):
    records_R = [record for table, record in values if table == 'R']  # get R entries
    records_S = [record for table, record in values if table == 'S']  # get S entries
    for record_R in records_R:
        for record_S in records_S:
            yield (record_R | record_S)  # return the natural join of the entries

table_R = [
    {'id': 1, 'name': 'John', 'age': 30},
    {'id': 2, 'name': 'Alice', 'age': 25},
    {'id': 3, 'name': 'Bob', 'age': 35}
]

table_S = [
    {'id': 1, 'city': 'New York'},
    {'id': 2, 'city': 'Los Angeles'},
    {'id': 4, 'city': 'Chicago'}
]

combined_data = table_R + table_S
output = MapReduce(lambda: [(None, record) for record in combined_data], MAP, REDUCE)
result = list(output)
result

[{'id': 1, 'name': 'John', 'age': 30, 'city': 'New York'},
 {'id': 2, 'name': 'Alice', 'age': 25, 'city': 'Los Angeles'}]

In [85]:
def MAP(_, record):
    record_id = record[0]  # get id from tuple
    if len(record) == 3:  # if from R
        yield (record_id, ('R', record))
    elif len(record) == 2:  # if from S
        yield (record_id, ('S', record))

def REDUCE(key, values):
    records_R = [record for table, record in values if table == 'R']
    records_S = [record for table, record in values if table == 'S']
    for record_R in records_R:
        for record_S in records_S:
            yield (record_R + record_S[1:])

def RECORDREADER():
    for record in table_R:
        yield (None, record)

    for record in table_S:
        yield (None, record)

table_R = [
    (1, 'John', 30),
    (2, 'Alice', 25),
    (3, 'Bob', 35)
]

table_S = [
    (1, 'New York'),
    (2, 'Los Angeles'),
    (4, 'Chicago')
]

output = MapReduce(RECORDREADER, MAP, REDUCE)

result = list(output)

result

[(1, 'John', 30, 'New York'), (2, 'Alice', 25, 'Los Angeles')]

In [86]:
from typing import NamedTuple

class RecordR(NamedTuple):
    id: int
    name: str
    age: int

class RecordS(NamedTuple):
    id: int
    city: str

def MAP(_, record):
    record_id = record[0] 
    if len(record) == 3:
        yield (record_id, ('R', record))
    elif len(record) == 2:
        yield (record_id, ('S', record))

def REDUCE(key, values):
    records_R = [record for table, record in values if table == 'R']
    records_S = [record for table, record in values if table == 'S']
    for record_R in records_R:
        for record_S in records_S:
            yield (record_R + record_S[1:])

def RECORDREADER():
    # Передача записей из таблицы R
    for record in table_R:
        yield (None, record)

    # Передача записей из таблицы S
    for record in table_S:
        yield (None, record)

table_R = [
    RecordR(id=1, name='John', age=30),
    RecordR(id=2, name='Alice', age=25),
    RecordR(id=3, name='Bob', age=35)
]

table_S = [
    RecordS(id=1, city='New York'),
    RecordS(id=2, city='Los Angeles'),
    RecordS(id=4, city='Chicago')
]

output = MapReduce(RECORDREADER, MAP, REDUCE)
result = list(output)

result

[(1, 'John', 30, 'New York'), (2, 'Alice', 25, 'Los Angeles')]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [87]:

def MAP(record):
    # create (a, b) pair for each tuple (a, b, c)
    a, b, _ = record
    yield (a, b)

def REDUCE(key, values):
    numeric_values = [value for value in values if isinstance(value, int)]
    result = sum(numeric_values)
    yield (key, result)

def RECORDREADER():
    for record in data:
        yield (record,)

data = [
    (1, 10, 20),
    (2, 15, 25),
    (1, 5, 30),
    (2, 20, 35)
]

output = MapReduce(RECORDREADER, MAP, REDUCE)
result = list(output)
print(result)

[(1, 15), (2, 35)]


In [88]:
from typing import NamedTuple

class DataRecord(NamedTuple):
    a: int
    b: int
    c: int


data = [
    DataRecord(a=1, b=10, c=20),
    DataRecord(a=2, b=15, c=25),
    DataRecord(a=1, b=5, c=30),
    DataRecord(a=2, b=20, c=35)
]


def MAP(record):
    yield (record[0].a, record[0].b)


def REDUCE(key, values):
    numeric_values = [value for value in values if isinstance(value, int)]
    result = sum(numeric_values)
    yield (key, result)

def RECORDREADER():
    for record in data:
        yield (record,)

def MapReduce(record_reader, mapper, reducer):
    intermediate = {}
    for record in record_reader():
        for mapped_key, mapped_value in mapper(record):
            intermediate.setdefault(mapped_key, []).append(mapped_value)
    for key, values in intermediate.items():
        for reduced in reducer(key, values):
            yield reduced

output = MapReduce(RECORDREADER, MAP, REDUCE)
result = list(output)
print(result)

[(1, 15), (2, 35)]


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [89]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [90]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(J):
    for k in range(K):
      yield ((j, k), big_mat[j, k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(I):
    yield ((i, k), small_mat[i, j] * w)

def REDUCE(key, values):
  (i, k) = key
  yield ((i, k), sum(values))

# self-check
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def as_matrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, as_matrix(solution))

True

In [91]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [92]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def RECORDREADER():
  for i in range(I):
    for j in range(J):
      yield ((0, i, j), small_mat[i, j])

  for j in range(J):
    for k in range(K):
      yield ((1, j, k), big_mat[j, k])

def MAP(k1, v1):
    (matrix_index, row_index, col_index) = k1
    yield (col_index, (matrix_index, row_index, v1)) if matrix_index == 0 else (row_index, (matrix_index, col_index, v1))


def REDUCE(key, values):
  first_matrix = [el for el in values if el[0] == 0]
  second_matrix = [el for el in values if el[0] == 1]

  for first_el in first_matrix:
    for second_el in second_matrix:
      (_, first_index, first_value) = first_el
      (_, second_index, second_value) = second_el
      yield ((first_index, second_index), first_value * second_value)


def MAP_IDENTITY(*args):
  yield args

def REDUCE_AGREGATE(key, values):
  yield (key, sum(values))

def GET_RECORDREADER(iterable):
  def func():
    for el in iterable:
      yield el
  return func

# self-check
reference_solution = np.matmul(small_mat, big_mat)
partial_solution = MapReduce(RECORDREADER, MAP, REDUCE)
solution = MapReduce(GET_RECORDREADER(partial_solution), MAP_IDENTITY, REDUCE_AGREGATE)

def as_matrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i, k), vw) in reduce_output) + 1
  K = max(k for ((i, k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I, K))
  for ((i, k), vw) in reduce_output:
    mat[i, k] = vw
  return mat

np.allclose(reference_solution, as_matrix(solution))

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [93]:
import numpy as np

# matrices' dims
I = 2
J = 3
K = 4 * 10

# generate random matrices
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def RECORDREADER():
    for i in range(I):
        for j in range(J):
            yield ((0, i, j), small_mat[i, j])  # ind and val of the elem of the small matrix

    for j in range(J):
        for k in range(K):
            yield ((1, j, k), big_mat[j, k])  # same for the big matrix

def MAP(k1, v1):
    (matrix_index, row_index, col_index) = k1  # get indices
    if matrix_index == 0:
        yield (col_index, (matrix_index, row_index, v1))
    else:
        yield (row_index, (matrix_index, col_index, v1))

def REDUCE(key, values):
    first_matrix = [el for el in values if el[0] == 0]
    second_matrix = [el for el in values if el[0] == 1]
    for first_el in first_matrix:
        for second_el in second_matrix:
            (_, first_index, first_value) = first_el  # get ind and val of the first elem
            (_, second_index, second_value) = second_el  # same for the second
            yield ((first_index, second_index), first_value * second_value)

def MAP_IDENTITY(*args):
    yield args

def REDUCE_AGREGATE(key, values):
    yield (key, sum(values))

def GET_RECORDREADER(iterable):
    def func():
        for el in iterable:
            yield el
    return func

reference_solution = np.matmul(small_mat, big_mat)

partial_solution = MapReduce(RECORDREADER, MAP, REDUCE)
solution = MapReduce(GET_RECORDREADER(partial_solution), MAP_IDENTITY, REDUCE_AGREGATE)

def as_matrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I, K))
    for ((i, k), vw) in reduce_output:
        mat[i, k] = vw
    return mat

np.allclose(reference_solution, as_matrix(solution))

True